In [24]:
import pandas as pd
url = 'https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv'
df = pd.read_csv(url)
print(df.head)

<bound method NDFrame.head of      Unnamed: 0     id                domain        type  \
0             0    141               awm.com  unreliable   
1             1    256     beforeitsnews.com        fake   
2             2    700           cnnnext.com  unreliable   
3             3    768               awm.com  unreliable   
4             4    791  bipartisanreport.com   clickbait   
..          ...    ...                   ...         ...   
245         245  39259     beforeitsnews.com        fake   
246         246  39468     beforeitsnews.com        fake   
247         247  39477       www.newsmax.com         NaN   
248         248  39550       www.newsmax.com         NaN   
249         249  39558       www.newsmax.com         NaN   

                                                   url  \
0    http://awm.com/church-congregation-brings-gift...   
1    http://beforeitsnews.com/awakening-start-here/...   
2    http://www.cnnnext.com/video/18526/never-hike-...   
3    http://awm.c

In [ ]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Ensure required NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

def clean_text(text: str) -> str:
    """
    Cleans a text string by replacing dates, emails, URLs, and numbers
    with placeholder tokens.
    """
    # Replace dates in formats like YYYY-MM-DD and DD-MM-YYYY
    text = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '<DATE>', text)
    text = re.sub(r'\b\d{2}-\d{2}-\d{4}\b', '<DATE>', text)
    # Replace email addresses
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '<EMAIL>', text)
    # Replace URLs (http/https)
    text = re.sub(r'(https?://\S+)', '<URL>', text)
    # Replace numbers (including decimals)
    text = re.sub(r'\b\d+(\.\d+)?\b', '<NUM>', text)
    
    return text.strip().lower()

def tokenize_text(text: str, stop_words: set) -> list:
    """
    Tokenizes the input text using NLTK and removes stopwords.
    """
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

def stem_tokens(tokens: list) -> list:
    """
    Applies Porter stemming to a list of tokens.
    """
    ps = PorterStemmer()
    return [ps.stem(token) for token in tokens]

def main():
    # Read the CSV file into a DataFrame and select the first 300 rows
    df = pd.read_csv("Datasets\\news_sample.csv", encoding="utf-8")
    df_sample = df.head(300)
    
    # Apply cleaning to every cell in the DataFrame
    df_clean = df_sample.applymap(lambda x: clean_text(str(x)))
    
    # Define the English stopwords set
    stop_words = set(stopwords.words('english'))
    
    # Tokenize each cell and remove stopwords, then join tokens back into a string
    df_tokens = df_clean.map(lambda x: " ".join(tokenize_text(x, stop_words)))
    
    # Save the tokenized data to CSV
    df_tokens.to_csv("block_stopwords_news_sample.csv", index=False)
    
    # Count word frequencies in the tokenized DataFrame
    all_tokens = []
    for cell in df_tokens.values.flatten():
        all_tokens.extend(cell.split())
    token_freq = Counter(all_tokens)
    
    # Count word frequencies in the original (uncleaned) sample
    all_original_words = []
    for cell in df_sample.map(lambda x: str(x).strip().lower()).values.flatten():
        all_original_words.extend(cell.split())
    original_freq = Counter(all_original_words)
    
    print(f"Word frequency in tokenized sample:\n{token_freq}\n")
    print(f"Word frequency in original sample:\n{original_freq}\n")
    
    # Apply stemming to each tokenized cell
    df_stem = df_tokens.map(lambda x: " ".join(stem_tokens(x.split())))
    df_stem.to_csv("stemmed_news_sample.csv", index=False)
    
    # Count word frequencies in the stemmed DataFrame
    all_stemmed_tokens = []
    for cell in df_stem.values.flatten():
        all_stemmed_tokens.extend(cell.split())
    stem_freq = Counter(all_stemmed_tokens)
    
    print(f"Word frequency in stemmed sample:\n{stem_freq}")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\nikla\AppData\Local\Temp\ipykernel_16780\461607894.py:51: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean = df_sample.applymap(lambda x: clean_text(str(x)))
C:\Users\nikla\AppData\Local\Temp\ipykernel_16780\461607894.py:57: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_tokens = df_clean.applymap(lambda x: " ".join(tokenize_text(x, stop_words)))
C:\Users\nikla\AppData\Local\Temp\ipykernel_16780\461607894.py:70: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for cell in df_sample.applymap(lambda x: str(x).strip().lower()).values.flatten():
C:\Users\nik

Word frequency in tokenized sample:
Counter({',': 11040, '.': 7319, '>': 6624, '<': 6618, 'num': 5399, ':': 2644, '’': 2335, 'nan': 1011, '“': 1001, '”': 943, 'date': 767, '(': 727, ')': 727, '?': 512, 'url': 460, '!': 458, 'one': 440, 'trump': 407, '``': 391, 'people': 387, '–': 387, '[': 336, ']': 336, 'like': 329, 'would': 328, "'": 305, 'us': 302, 'time': 295, 'new': 293, 'also': 276, ';': 269, 'blockchain': 260, 'said': 237, '&': 235, 'many': 229, 'market': 225, 'world': 222, 'next': 212, 'even': 212, 'government': 212, 'could': 211, 'president': 209, 'state': 208, 'think': 199, "'s": 193, 'way': 191, 'may': 190, 'see': 187, 'two': 186, 'first': 185, '$': 183, "''": 179, 'life': 178, 'fake': 173, 'u.s.': 171, 'make': 168, '%': 166, 'things': 160, 'obama': 159, 'get': 157, 'take': 157, 'beforeitsnews.com': 155, 'free': 152, 'war': 148, 'years': 148, 'bitcoin': 147, 'much': 144, 'states': 144, '—': 143, 'stocks': 141, '-': 141, 'fact': 138, 'going': 137, 'russia': 136, 'news': 136, 

We're using Pandas and NLTK because Pandas offers a robust data handling and cleaning capabilities, which is essential for large datasets. While Provides a suite of NLP tools (tokenization, stopwords, stemming) that are standard in text preprocessing. These operations help normalize the data for downstream analysis and modeling.

Pandas DataFrame: It’s flexible, easy to filter and analyze, and supports integration with visualization libraries.
Rationale: A DataFrame structure is ideal for handling large datasets and performing exploratory data analysis (EDA).
